In [3]:
import os
import pandas as pd
import numpy as np
os.chdir('/home/peisenha/restudToolbox/package/development/testing/property')



In [24]:
data_frame = pd.read_pickle('debug.respy.pkl')
num_missing = int(np.sum(data_frame['Choice'].isin([1, 2])) * 0.5)
index_missing = np.random.choice(data_frame['Earnings'][is_working].index, num_missing, replace=False)
data_frame.loc[index_missing, 'Earnings'] = None

print data_frame

   Identifier  Period  Choice  Earnings  Experience A  Experience B  \
0           0       0       2       NaN             0             0   
1           1       0       3       NaN             0             0   
3           3       0       2      0.91             0             0   
2           2       0       1      1.13             0             0   

   Years Schooling  Lagged Schooling  
0                5                 1  
1                5                 1  
3                5                 1  
2                5                 1  
